# Load CSV Files into Postgres Database

-Before running this notebook, create a new "crowdfunding_db" databse in Postgres, then load the schema.sql file to create the tables

In [1]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, text, inspect, func

# data
import pandas as pd
import numpy as np

# viz
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Load variables with connection information to database
SQL_USERNAME = "postgres"
SQL_PASSWORD = "postgres"
SQL_IP = "localhost"
PORT = 5432
DATABASE = "crowdfunding_db"

In [3]:
# create connection using pre-defined variables
connection_string = f"postgresql+psycopg2://{SQL_USERNAME}:{SQL_PASSWORD}@{SQL_IP}:{PORT}/{DATABASE}"
engine = create_engine(connection_string)

In [4]:
# explore and understand the data

# Create the inspector and connect it to the engine
inspector = inspect(engine)

# Collect the names of tables within the database
tables = inspector.get_table_names()

# Using the inspector to print the column names within each table and its types
for table in tables:
    print(table)
    columns = inspector.get_columns(table)
    for column in columns:
        print(column["name"], column["type"])
        
    print()

contacts
contact_id INTEGER
first_name VARCHAR(25)
last_name VARCHAR(25)
email VARCHAR(50)
last_updated TIMESTAMP

campaign
cf_id INTEGER
contact_id INTEGER
company_name VARCHAR(50)
description VARCHAR(100)
goal DOUBLE PRECISION
pledged DOUBLE PRECISION
outcome VARCHAR(25)
backers_count INTEGER
country VARCHAR(5)
currency VARCHAR(5)
launched_date TIMESTAMP
end_date TIMESTAMP
category_id VARCHAR(10)
subcategory_id VARCHAR(10)
last_updated TIMESTAMP

category
category_id VARCHAR(10)
category VARCHAR(50)
last_updated TIMESTAMP

subcategory
subcategory_id VARCHAR(10)
subcategory VARCHAR(50)
last_updated TIMESTAMP



In [5]:
# load campaign data from csv into new dataframe
campaign_df = pd.read_csv("Resources/campaign.csv")
# print shape to identify number of rows in dataframe
print(campaign_df.shape)
campaign_df.head()

(1000, 14)


,cf_id,contact_id,company_name,description,goal,pledged,outcome,backers_count,country,currency,launched_date,end_date,category_id,subcategory_id
0,147,4661,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100.0,0.0,failed,0,CA,CAD,2020-02-13,2021-03-01,cat1,subcat1
1,1621,3765,Odom Inc,Managed bottom-line architecture,1400.0,14560.0,successful,158,US,USD,2021-01-25,2021-05-25,cat2,subcat2
2,1812,4187,"Melton, Robinson and Fritz",Function-based leadingedge pricing structure,108400.0,142523.0,successful,1425,AU,AUD,2020-12-17,2021-12-30,cat3,subcat3
3,2156,4941,"Mcdonald, Gonzalez and Ross",Vision-oriented fresh-thinking conglomeration,4200.0,2477.0,failed,24,US,USD,2021-10-21,2022-01-17,cat2,subcat2
4,1365,2199,Larson-Little,Proactive foreground core,7600.0,5265.0,failed,53,US,USD,2020-12-21,2021-08-23,cat4,subcat4


In [6]:
# load category data from csv into new dataframe
category_df = pd.read_csv("Resources/category.csv")
# print shape to identify number of rows in dataframe
print(category_df.shape)
category_df.head()

(9, 2)


,category_id,category
0,cat1,food
1,cat2,music
2,cat3,technology
3,cat4,theater
4,cat5,film & video


In [7]:
# load contacts data from csv into new dataframe
contacts_df = pd.read_csv("Resources/contacts.csv")
# print shape to identify number of rows in dataframe
print(contacts_df.shape)
contacts_df.head()

(1000, 4)


,contact_id,first_name,last_name,email
0,4661,Cecilia,Velasco,cecilia.velasco@rodrigues.fr
1,3765,Mariana,Ellis,mariana.ellis@rossi.org
2,4187,Sofie,Woods,sofie.woods@riviere.com
3,4941,Jeanette,Iannotti,jeanette.iannotti@yahoo.com
4,2199,Samuel,Sorgatz,samuel.sorgatz@gmail.com


In [8]:
# load subcategory data from csv into new dataframe
subcategory_df = pd.read_csv("Resources/subcategory.csv")
# print shape to identify number of rows in dataframe
print(subcategory_df.shape)
subcategory_df.head()

(24, 2)


,subcategory_id,subcategory
0,subcat1,food trucks
1,subcat2,rock
2,subcat3,web
3,subcat4,plays
4,subcat5,documentary


In [9]:
# Use pandas and sqlalchemy to insert category dataframe into crowdfunding_db - category table
category_df.to_sql("category", schema="public", con=engine, index=False, if_exists="append", method="multi")

9

In [10]:
# Use pandas and sqlalchemy to insert category dataframe into crowdfunding_db - subcategory table
subcategory_df.to_sql("subcategory", schema="public", con=engine, index=False, if_exists="append", method="multi")

24

In [11]:
# Use pandas and sqlalchemy to insert category dataframe into crowdfunding_db - contacts table
contacts_df.to_sql("contacts", schema="public", con=engine, index=False, if_exists="append", method="multi")

1000

In [12]:
# Use pandas and sqlalchemy to insert category dataframe into crowdfunding_db - campaign table
# campaign data gets inserted last because it contains foreign key columns dependent on the tables above
campaign_df.to_sql("campaign", schema="public", con=engine, index=False, if_exists="append", method="multi")

1000